1. data 로드를 통한 데이터 프레임 완성
2. 한글에 맞게 형태소 분석기를 이용하여 단어 토큰화 작업후 TF-IDF를 계산
3. 코사인 유사도 기반을 통한 영화 3종 추천

In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('data2.csv',low_memory=False)

In [3]:
tg=Okt()
def tk_f(a):
    tk_all_d= []
    sum_d = ''
    for i in range(len(a)):
        tk_d = [x for x in tg.nouns(a[i])]
        for i in tk_d:
            sum_d = sum_d + ' '+ i
        tk_all_d.append(sum_d)
        sum_d = ''
    return tk_all_d
tk_data = tk_f(data['content'])
tk_data
data['tk_data']=tk_data
data

,name,genre,content,tk_data
0,올드보이,스릴러,술 좋아하고 떠들기 좋아하는 오.대.수. 본인의 이름풀이를 오늘만 대충 수습하며 살...,술 오 수 본인 이름 풀이 오늘 대충 수습 이 남자 아내 딸아이 샐러리맨 날 술 ...
1,노트북,로맨스,"17살, ‘노아’는 밝고 순수한 ‘앨리’를 보고 첫눈에 반한다. 빠른 속도로 서로에...",살 노아 앨리 를 보고 눈 속도 서로 둘 이 앞 장벽 이별 살 앨리 신문 노아 의...
2,미션임파서블,액션,"도난당한 핵무기를 되찾아 오는 위험천만한 작전 중, 에단 헌트(톰 크루즈)는 임무를...",도난 핵무기 위험천만 작전 중 에단 헌트 톰 크루즈 임무 포기 팀 목숨 구 선택 ...
3,니모를찾아서,애니메이션,“디즈니`픽사 최고의 애니메이션이 3D로 다시 찾아온다!” 호기심 가득한 아기 물고...,디즈니 픽사 최고 애니메이션 로 다시 호기심 아기 물고기 니모 인간 납치 아들 바...
4,탑건,액션,해군 최신 전투기 F-14기를 모는 젊은 조종사 매버릭 대위(톰 크루즈)는 최고의 ...,해군 최신 전투기 모 조종사 매버릭 대위 톰 크루즈 최고 실력 자부 파일럿 최 정...
5,친절한금자씨,스릴러,주변 사람들의 시선을 단번에 사로잡을 만큼 뛰어난 미모의 소유자인 금자(이영애)는 ...,주변 사람 시선 단번 만큼 미모 소유자 금 이영애 스무 살 죄 감옥 가게 나이 외...
6,반지의제왕,판타지,모든 힘을 지배할 악의 군주 ‘사우론’의 절대반지가 깨어나고 악의 세력이 세상을 지...,모든 힘 지배 악의 군주 사우론 의 절대반지 악의 세력 세상 지배 중간계 혼란 처...
7,신세계,범죄,경찰청 수사 기획과 강과장(최민식)은 국내 최대 범죄 조직인 '골드문'이 기업형 조...,경찰청 수사 기획 강 과장 최민식 은 국내 최대 범죄 조직 골드 문 이 기업 조직...
8,아저씨,NaN,불행한 사건으로 아내를 잃고 세상을 등진 채 전당포를 꾸려가며 외롭게 살아가는 전직...,사건 아내 세상 등진 채 전당포 전직 특수요원 태식 사람 전당포 물건 사람 옆집 ...
9,해리포터,NaN,해리 포터(다니엘 래드클리프 분)는 위압적인 버논 숙부(리챠드 그리피스 분)와 냉담...,해리 포터 다니엘 래드클리프 분 위압 버논 숙부 리 챠드 그리피스 분 이모 페투니...


In [4]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_t = tfidf.fit_transform(data['tk_data'])
cos = cosine_similarity(tfidf_t,tfidf_t)
t_idx = dict(zip(data['name'],data.index))
def ck_s_t_cos(t,sim=cos):
    idx = t_idx[t] 
    c_sc = list(enumerate(sim[idx])) 
    c_sc = sorted(c_sc,key = lambda x:x[1],reverse=True) #cos 수치가 높아야 유사도가 높다 (내림차순)
    m_i= c_sc[1:4]
    print(f'코사인유사도 1~3위 인덱스,수치 : {m_i}')
    m_idx = [i[0] for i in m_i]
    return data['name'].iloc[m_idx]
ck_s_t_cos('올드보이')

코사인유사도 1~3위 인덱스,수치 : [(5, 0.11062607540975497), (11, 0.09231554282001697), (14, 0.0863722422457267)]


5       친절한금자씨
11      어바웃 타임
14    지금만나러갑니다
Name: name, dtype: object

Q2. 다른 유사도 방식을 이용하여서도 같은 결과가 나오는지 검토

In [5]:
# from sklearn.metrics import jaccard_score
from sklearn.metrics import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances

In [6]:
euc = euclidean_distances(tfidf_t,tfidf_t)
def ck_s_t_euc(t,sim=euc):
    idx = t_idx[t] 
    c_sc = list(enumerate(sim[idx])) 
    c_sc = sorted(c_sc,key = lambda x:x[1]) #거리가 짧을수록 유사도가 높다 (오름차순)
    m_i= c_sc[1:4]
    print(f'유클리드유사도 1~3위 인덱스,수치 : {m_i}')
    m_idx = [i[0] for i in m_i]
    return data['name'].iloc[m_idx]

In [7]:
man = manhattan_distances(tfidf_t,tfidf_t)
def ck_s_t_man(t,sim=man):
    idx = t_idx[t] 
    c_sc = list(enumerate(sim[idx])) 
    c_sc = sorted(c_sc,key = lambda x:x[1]) #거리가 짧을수록 유사도가 높다 (오름차순)
    m_i= c_sc[1:4]
    print(f'멘하탄유사도 1~3위 인덱스,수치 : {m_i}')
    m_idx = [i[0] for i in m_i]
    return data['name'].iloc[m_idx]

In [9]:
print(ck_s_t_cos('올드보이'))
print(ck_s_t_euc('올드보이'))
print(ck_s_t_man('올드보이'))

코사인유사도 1~3위 인덱스,수치 : [(5, 0.11062607540975497), (11, 0.09231554282001697), (14, 0.0863722422457267)]
5       친절한금자씨
11      어바웃 타임
14    지금만나러갑니다
Name: name, dtype: object
유클리드유사도 1~3위 인덱스,수치 : [(5, 1.3336970604978071), (11, 1.3473562685347804), (14, 1.3517601545794091)]
5       친절한금자씨
11      어바웃 타임
14    지금만나러갑니다
Name: name, dtype: object
멘하탄유사도 1~3위 인덱스,수치 : [(14, 11.109437544466976), (1, 11.42810509315834), (10, 11.796278722355686)]
14    지금만나러갑니다
1          노트북
10          장고
Name: name, dtype: object
